In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

geojson_file = '/content/drive/MyDrive/CS 411/GeoJSON files/Apartments.geojson'  # Path to your GeoJSON file
csv_file = 'Apartments.csv'  # Output CSV file
specific_fields = ['Address']  # Fields you want to keep

# Read the GeoJSON file
gdf = gpd.read_file(geojson_file)
gdf = gdf.to_crs(epsg=4326)

# Calculate the centroids of the geometries
print(gdf.iloc[0].geometry)
gdf['centroid'] = gdf.geometry.centroid

# Extract latitude and longitude from centroids
gdf['latitude'] = gdf['centroid'].y
gdf['longitude'] = gdf['centroid'].x

# Create a DataFrame with specific fields and centroid coordinates
df = gdf[specific_fields + ['latitude', 'longitude']]

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)
print(f"Conversion completed. The CSV file is saved as {csv_file}.")

POLYGON ((-88.25315115647089 40.14016619247847, -88.2531514870202 40.14019446048516, -88.25311645609855 40.14019470190791, -88.25311691881356 40.140234306117144, -88.25312038998868 40.14023428240352, -88.2531211246763 40.14029707417753, -88.25309842706466 40.140297230472335, -88.25309882365738 40.140331186843646, -88.25313995679124 40.140330903720454, -88.25314117244498 40.14043477312888, -88.25320129506791 40.1404343590329, -88.2532014901919 40.140451062976844, -88.25314324514007 40.140451463963146, -88.25314603997836 40.14069037440131, -88.25320493300408 40.140689968458304, -88.25320511212908 40.14070530885102, -88.25314400939898 40.1407057294752, -88.25314523247435 40.140810283353545, -88.25310074167626 40.14081058936963, -88.25310108495441 40.14083999216388, -88.2531247778723 40.140839829353084, -88.25312556150162 40.14090691441212, -88.25311959778072 40.14090695531243, -88.253120086077 40.14094876065083, -88.25315986301814 40.14094848648812, -88.25316018080403 40.14097563861339, -

<ipython-input-13-a9bfa7577696>:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random

# Load GeoJSON data
geojson_file = '/content/drive/MyDrive/CS 411/GeoJSON files/Apartments.geojson'
csv_file = 'Apartments_with_block.csv'
specific_fields = ['address', 'safestay_score']

# Read the GeoJSON file
gdf = gpd.read_file(geojson_file)
gdf = gdf.to_crs(epsg=4326)  # Project to a metric CRS (e.g., EPSG:3857)

# Calculate centroids
gdf['centroid'] = gdf.geometry.centroid
gdf['latitude'] = gdf['centroid'].y
gdf['longitude'] = gdf['centroid'].x

# Get the bounding box (min_x, min_y, max_x, max_y)
minx, miny, maxx, maxy = gdf.total_bounds
print(f"Bounding box: {minx}, {miny}, {maxx}, {maxy}")

# Define the number of grid cells in the x and y direction
num_cells_x = 10  # Adjust as needed
num_cells_y = 10  # Adjust as needed

# Calculate the size of each cell
cell_width = (maxx - minx) / num_cells_x
cell_height = (maxy - miny) / num_cells_y

# Assign each apartment to a grid cell
def assign_grid_index(row, minx, miny, cell_width, cell_height, num_cells_x):
    x_idx = int((row['longitude'] - minx) // cell_width)
    y_idx = int((row['latitude'] - miny) // cell_height)

    # Ensure that indices don't exceed the grid limits
    x_idx = min(x_idx, num_cells_x - 1)
    y_idx = min(y_idx, num_cells_y - 1)

    # Convert the 2D index to a row-major 1D index
    grid_index = y_idx * num_cells_x + x_idx
    return grid_index

# Apply the function to calculate the grid index for each apartment
gdf['block'] = gdf.apply(assign_grid_index, axis=1,
                              args=(minx, miny, cell_width, cell_height, num_cells_x))

# Create a DataFrame with specific fields, centroid coordinates, and grid index
gdf = gdf.drop_duplicates(subset=['Address'], keep='first')

gdf['safestay_score'] = gdf.apply(lambda row: random.randint(0, 10), axis=1)

gdf['address'] = gdf['Address']

apartments_gdf = gdf

df = gdf[specific_fields + ['latitude', 'longitude', 'block']]

# Save the result to a CSV file
df.to_csv(csv_file, index=False)
print(f"Conversion completed. The CSV file with grid indices is saved as {csv_file}.")

Bounding box: -88.33291343266208, 40.064900796278714, -88.22903384495362, 40.17137116087465
Conversion completed. The CSV file with grid indices is saved as Apartments_with_block.csv.


<ipython-input-16-862eb408a28d>:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [ ]:
geojson_file = '/content/drive/MyDrive/CS 411/GeoJSON files/Streetlights.geojson'  # Path to your GeoJSON file
csv_file = 'Streetlights_with_block.csv'  # Output CSV file
specific_fields = ['streetlight_id', 'pole_material', 'wattage', 'height', 'color']  # Fields you want to keep

# Read the GeoJSON file
gdf = gpd.read_file(geojson_file)
gdf = gdf.to_crs(epsg=4326)

# Calculate the centroids of the geometries
gdf['centroid'] = gdf.geometry.centroid

# Extract latitude and longitude from centroids
gdf['latitude'] = gdf['centroid'].y
gdf['longitude'] = gdf['centroid'].x

gdf['block'] = gdf.apply(assign_grid_index, axis=1,
                              args=(minx, miny, cell_width, cell_height, num_cells_x))
gdf['streetlight_id'] = gdf['OBJECTID']
gdf['wattage'] = gdf['Lamp_Wattage'].str.extract('(\d+)')
gdf['wattage'] = pd.to_numeric(gdf['wattage'], errors='coerce')
gdf['pole_material'] = gdf['Pole_Material']
gdf['height'] = gdf['Pole_Height']
gdf['color'] = gdf['Pole_Color']

# Create a DataFrame with specific fields and centroid coordinates
df = gdf[specific_fields + ['latitude', 'longitude', 'block']]

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)
print(f"Conversion completed. The CSV file is saved as {csv_file}.")

Conversion completed. The CSV file is saved as Streetlights_with_block.csv.


<ipython-input-18-b2e64c094f6f>:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [ ]:
geojson_file = '/content/drive/MyDrive/CS 411/GeoJSON files/pedestrian_crashes.json'  # Path to your GeoJSON file
csv_file = 'Crash_with_block.csv'  # Output CSV file
specific_fields = ['crash_id', 'crash_severity', 'traffic_control', 'year', 'road_surface']  # Fields you want to keep

# Read the GeoJSON file
gdf = gpd.read_file(geojson_file)
gdf = gdf.to_crs(epsg=4326)

# Calculate the centroids of the geometries
gdf['centroid'] = gdf.geometry.centroid

# Extract latitude and longitude from centroids
gdf['latitude'] = gdf['centroid'].y
gdf['longitude'] = gdf['centroid'].x

gdf['block'] = gdf.apply(assign_grid_index, axis=1,
                              args=(minx, miny, cell_width, cell_height, num_cells_x))
gdf['crash_id'] = range(1, len(gdf) + 1)

# Create a DataFrame with specific fields and centroid coordinates
df = gdf[specific_fields + ['latitude', 'longitude', 'block']]

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)
print(f"Conversion completed. The CSV file is saved as {csv_file}.")

Conversion completed. The CSV file is saved as Crash_with_block.csv.


<ipython-input-19-a28406945ff5>:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [ ]:
csv_file = 'Location.csv'  # Output CSV file
df = pd.DataFrame({
    'block': np.arange(100),
    'has_police_station': np.random.choice([0, 1], size=100)
})
df.to_csv(csv_file, index=False)
print(f"Conversion completed. The CSV file is saved as {csv_file}.")

Conversion completed. The CSV file is saved as Location.csv.


In [ ]:
import pandas as pd
import random
import string

csv_file = 'Users.csv'

# Helper functions to generate random data
def random_string(length):
    return ''.join(random.choices(string.ascii_letters, k=length))

def generate_users(num_users):
    users = []
    for i in range(num_users):
        username = f"user{i+1}"
        password = random_string(10)
        first_name = random_string(7)
        last_name = random_string(7)
        users.append([username, password, first_name, last_name])
    return users

# Create the users DataFrame
num_users = 500
user_df = pd.DataFrame(generate_users(num_users), columns=['username', 'password', 'first_name', 'last_name'])
user_df.head()

user_df.to_csv(csv_file, index=False)

In [ ]:
csv_file = 'Ratings.csv'

# Assuming gdf contains the list of apartments with addresses
apartment_addresses = apartments_gdf['address'].tolist()

# Helper function to generate random ratings for each user
def generate_ratings(user_df, apartment_addresses, ratings_per_user):
    ratings = []
    for _, row in user_df.iterrows():
        username = row['username']
        addresses_sample = random.sample(apartment_addresses, ratings_per_user)
        for address in addresses_sample:
            rating = random.randint(0, 10)  # Ratings between 1 and 5
            ratings.append([username, address, rating])
    return ratings

# Create the ratings DataFrame
ratings_per_user = 10
rating_df = pd.DataFrame(generate_ratings(user_df, apartment_addresses, ratings_per_user), columns=['username', 'address', 'rating'])
rating_df.head()

rating_df.to_csv(csv_file, index=False)

In [ ]:
len(rating_df)

5000